# 高度なファインチューニング実習

このノートブックでは、LLMのファインチューニングの高度なテクニックを学習します。

## 学習目標
- カスタムデータセットでのファインチューニング
- 異なるファインチューニング手法の比較
- ハイパーパラメータの最適化
- モデルの評価と分析
- 実用的なアプリケーションへの応用

## 前提条件
- LLM基礎実習の完了
- PyTorchとTransformersの基本理解
- 機械学習の基礎知識

## 注意事項
- このノートブックは計算集約的です
- GPUの使用を強く推奨します
- 実行時間は数時間に及ぶ場合があります


## 1. 環境セットアップとライブラリのインポート


In [ ]:
# 必要なライブラリのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets accelerate evaluate scikit-learn pandas numpy matplotlib seaborn tqdm wandb
!pip install peft bitsandbytes  # LoRA用のライブラリ

# ライブラリのインポート
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset as HFDataset, load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, classification_report
import json
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 環境設定
plt.style.use('default')
sns.set_palette("husl")

print("✅ ライブラリのインポートが完了しました")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
